In [1]:
#Part 3: Spark Streaming

In [1]:
#from pyspark.sql.functions import col, expr, row_number, from_json
from pyspark.sql.functions import *
import time

In [2]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder.appName("cs544")
         .config("spark.sql.shuffle.partitions", 10)
         .config("spark.ui.showConsoleProgress", False)
         .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.2')
         .getOrCreate())

df = (
    spark.readStream.format("kafka")
    .option("kafka.bootstrap.servers", "kafka:9092")
    .option("subscribe", "stations-json")
    .option("startingOffsets", "earliest")
    .load()
)

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3e960f7e-8b8f-49f3-a5ba-91c8f2b9e7a3;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.2.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.2 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.ap

In [3]:
df.isStreaming

True

In [4]:
schema = "station STRING, date DATE, degrees DOUBLE, raining INTEGER"
stations_json = (df.select(
    col("key").cast("string"), 
    from_json(col("value").cast("string"), schema).alias("value")).select("key", "value.*"))

In [5]:
stations_json

DataFrame[key: string, station: string, date: date, degrees: double, raining: int]

In [6]:
stations_json.isStreaming

True

In [7]:
#Stats

In [8]:
counts_df = (stations_json.groupBy("station").agg(
    min("date").alias("start"),
    max("date").alias("end"),
    count("degrees").alias("measurements"),
    avg("degrees").alias("avg_temp"),
    max("degrees").alias("max_temp")
).orderBy("station"))

s = counts_df.writeStream.format("console").trigger(processingTime="5 seconds").outputMode("complete").start()
s.awaitTermination(30)
s.stop()

23/04/28 21:02:01 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-da81244b-af47-46cf-bf39-388097d664fa. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/04/28 21:02:01 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+-------+----------+----------+------------+------------------+-----------------+
|station|     start|       end|measurements|          avg_temp|         max_temp|
+-------+----------+----------+------------+------------------+-----------------+
|      A|2000-01-01|2000-04-19|         110| 32.80299305471417|68.03134015480184|
|      B|2000-01-01|2000-04-19|         110| 42.40158052349588|78.68175239288585|
|      C|2000-01-01|2000-04-19|         110| 40.89114937158351|67.22620383475424|
|      D|2000-01-01|2000-04-19|         110| 19.80893376198706|56.67195165651097|
|      E|2000-01-01|2000-04-19|         110|27.109123091357272|61.88725386538411|
|      F|2000-01-01|2000-04-19|         110| 39.70651580674823| 74.3620554541848|
|      G|2000-01-01|2000-04-19|         110| 30.83161194114009| 67.7774949347819|
|      H|2000-01-01|2000-04-19|         110| 36.76185850294383|60.30637273006978|
|

23/04/28 21:02:18 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 5000 milliseconds, but spent 16551 milliseconds


-------------------------------------------
Batch: 1
-------------------------------------------
+-------+----------+----------+------------+------------------+-----------------+
|station|     start|       end|measurements|          avg_temp|         max_temp|
+-------+----------+----------+------------+------------------+-----------------+
|      A|2000-01-01|2000-04-30|         121| 35.82021717566553|80.31178172940585|
|      B|2000-01-01|2000-04-30|         121| 44.10600713637261|78.68175239288585|
|      C|2000-01-01|2000-04-30|         121| 43.29001425950536|80.21680463474156|
|      D|2000-01-01|2000-04-30|         121| 22.24311688414182|56.67195165651097|
|      E|2000-01-01|2000-04-30|         121| 28.34007469332162|61.88725386538411|
|      F|2000-01-01|2000-04-30|         121| 41.92348786370325|76.31898672762796|
|      G|2000-01-01|2000-04-30|         121|32.434060816037054| 67.7774949347819|
|      H|2000-01-01|2000-04-30|         121|39.552857610911374|74.70794418018109|
|

23/04/28 21:02:31 WARN TaskSetManager: Lost task 4.0 in stage 20.0 (TID 148) (8ae47c622be4 executor driver): TaskKilled (Stage cancelled)
23/04/28 21:02:31 WARN TaskSetManager: Lost task 5.0 in stage 20.0 (TID 149) (8ae47c622be4 executor driver): TaskKilled (Stage cancelled)


In [9]:
# Define DataFrames named today and features that look like this:

# DataFrame[station: string, date: date, raining: int]
# DataFrame[station: string, date: date, month: int, sub1degrees: float, sub1raining: int, sub2degrees: float, 
# sub2raining: int]

schema = "station STRING, date DATE, degrees FLOAT, raining INTEGER"
today = (df.select(from_json(col("value").cast("string"), schema).alias("value"))
         .select("value.station", "value.date", "value.raining")
        )


sub1features = (df.select(from_json(col("value").cast("string"), schema).alias("value"))
            .select("value.station",
                    date_add("value.date", 1).alias("date"),
                    col("value.degrees").alias("sub1degrees"),
                    col("value.raining").alias("sub1raining")
                   )
            )
sub2features = (df.select(from_json(col("value").cast("string"), schema).alias("value"))
            .select("value.station",
                    date_add("value.date", 2).alias("date"),
                    col("value.degrees").alias("sub2degrees"),
                    col("value.raining").alias("sub2raining")
                   )
            )

features = (df.select(from_json(col("value").cast("string"), schema).alias("value"))
            .select("value.station","value.date", month("value.date").alias("month"))
                    .join(sub1features, ["station","date"], "inner")
                    .join(sub2features, ["station","date"], "inner")
                   
           )

#Join today with features on the date and station columns
Rain_Forecast = today.join(features, ["station","date"], "inner")

In [10]:
assert str(today) == "DataFrame[station: string, date: date, raining: int]"
assert str(features) == "DataFrame[station: string, date: date, month: int, sub1degrees: float, sub1raining: int, sub2degrees: float, sub2raining: int]"

In [11]:
Rain_Forecast

DataFrame[station: string, date: date, raining: int, month: int, sub1degrees: float, sub1raining: int, sub2degrees: float, sub2raining: int]

In [12]:
Rain_Forecast.isStreaming

True

In [13]:
#view final dataframe
# s = Rain_Forecast.writeStream.format("console").trigger(processingTime="5 seconds").outputMode("append").start()
# s.awaitTermination(30)
# s.stop()

In [14]:
resp = !hdfs dfs -cat hdfs://nn:9000/Rain_Forecast
if str(resp[0]) == "cat: `hdfs://nn:9000/Rain_Forecast': No such file or directory":
    print(resp[0])
    !hdfs dfs -mkdir hdfs://nn:9000/Rain_Forecast
    resp = !hdfs dfs -cat hdfs://nn:9000/Rain_Forecast
    assert resp[0] == "cat: `hdfs://nn:9000/Rain_Forecast': Is a directory"
    print("`hdfs://nn:9000/Rain_Forecast': Is now a directory")
else:
    print("`hdfs://nn:9000/Rain_Forecast': Is already a directory")

resp = !hdfs dfs -cat hdfs://nn:9000/Rain_Forecast_Checkpoint
if resp[0] == "cat: `hdfs://nn:9000/Rain_Forecast_Checkpoint': No such file or directory":
    print(resp[0])
    !hdfs dfs -mkdir hdfs://nn:9000/Rain_Forecast_Checkpoint
    resp = !hdfs dfs -cat hdfs://nn:9000/Rain_Forecast_Checkpoint
    assert resp[0] == "cat: `hdfs://nn:9000/Rain_Forecast_Checkpoint': Is a directory"
    print("`hdfs://nn:9000/Rain_Forecast_Checkpoint': Is now a directory")
else:
    print("`hdfs://nn:9000/Rain_Forecast_Checkpoint': Is already a directory")

cat: `hdfs://nn:9000/Rain_Forecast': No such file or directory
`hdfs://nn:9000/Rain_Forecast': Is now a directory
cat: `hdfs://nn:9000/Rain_Forecast_Checkpoint': No such file or directory
`hdfs://nn:9000/Rain_Forecast_Checkpoint': Is now a directory


In [15]:
query = (Rain_Forecast
 .repartition(1)
 .writeStream
 .format("parquet")
 .trigger(processingTime="1 minute")
 .option("path", "hdfs://nn:9000/Rain_Forecast") #HDFS directory that will accumulate parquet files
 .option("checkpointLocation", "hdfs://nn:9000/Rain_Forecast_Checkpoint") #HDFS directory where Spark stores info about how to supress duplicates when reading those parquet files
 .start())

23/04/28 21:03:14 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [16]:
#time.sleep(120)
!hdfs dfs -ls hdfs://nn:9000/Rain_Forecast

Found 3 items
drwxr-xr-x   - root supergroup          0 2023-04-28 21:04 hdfs://nn:9000/Rain_Forecast/_spark_metadata
-rw-r--r--   3 root supergroup      30510 2023-04-28 21:03 hdfs://nn:9000/Rain_Forecast/part-00000-15106b58-5ebb-46dc-af5a-58f52cd6a42e-c000.snappy.parquet
-rw-r--r--   3 root supergroup       8727 2023-04-28 21:04 hdfs://nn:9000/Rain_Forecast/part-00000-d8b196a8-03aa-46f2-a224-fbe6cb162676-c000.snappy.parquet


In [17]:
#Part 4: Spark ML

In [18]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [19]:
#Read your parquet files from part 3
df = spark.read.format("parquet").load("hdfs://nn:9000/Rain_Forecast")

In [20]:
df.isStreaming

False

In [21]:
#Use the VectorAssembler to combine the feature columns 
#("month", "sub1degrees", "sub1raining", "sub2degrees", "sub2raining") into a single column.
va = VectorAssembler(inputCols=["month", "sub1degrees", "sub1raining", "sub2degrees", "sub2raining"], outputCol="features")
data = va.transform(df)

In [22]:
#Split your data into train and test.
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)

In [23]:
#Train a DecisionTreeClassifier on your training data to predict raining based on the features.
dt_classifier = DecisionTreeClassifier(featuresCol="features", labelCol="raining", predictionCol="prediction", maxDepth=5)
dt_model = dt_classifier.fit(train_data)

In [24]:
print(dt_model.toDebugString)

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_079b4c3caa21, depth=5, numNodes=33, numClasses=2, numFeatures=5
  If (feature 2 <= 0.5)
   If (feature 0 <= 3.5)
    If (feature 0 <= 2.5)
     If (feature 3 <= 62.97582244873047)
      Predict: 0.0
     Else (feature 3 > 62.97582244873047)
      Predict: 1.0
    Else (feature 0 > 2.5)
     Predict: 0.0
   Else (feature 0 > 3.5)
    If (feature 0 <= 5.5)
     If (feature 3 <= 36.147695541381836)
      If (feature 3 <= 25.290410041809082)
       Predict: 1.0
      Else (feature 3 > 25.290410041809082)
       Predict: 0.0
     Else (feature 3 > 36.147695541381836)
      If (feature 1 <= 52.76169204711914)
       Predict: 1.0
      Else (feature 1 > 52.76169204711914)
       Predict: 0.0
    Else (feature 0 > 5.5)
     Predict: 0.0
  Else (feature 2 > 0.5)
   If (feature 3 <= 31.003357887268066)
    If (feature 0 <= 3.5)
     If (feature 0 <= 1.5)
      If (feature 1 <= 33.99040985107422)
       Predict: 1.0
      Else (feature 1

In [25]:
#Use the model to make predictions on the test data.
predictions = dt_model.transform(test_data)
evaluator = MulticlassClassificationEvaluator(labelCol="raining", predictionCol="prediction",metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

In [26]:
avg_raining = predictions.filter(predictions.raining == 1).count() / predictions.select("raining").count()

In [27]:
#What is the accuracy (percent of times the model is correct)?
print("+------------------+-------------------+")
print("|      avg(correct)|       avg(raining)|")
print("+------------------+-------------------+")
print(f"|{accuracy}|{avg_raining}|")
print("+------------------+-------------------+")

+------------------+-------------------+
|      avg(correct)|       avg(raining)|
+------------------+-------------------+
|0.7595541401273885|0.3471337579617834|
+------------------+-------------------+


In [28]:
#Model Deployment

In [29]:
schema = "station STRING, date DATE, raining INTEGER, month INTEGER, sub1degrees FLOAT, sub1raining INTEGER, sub2degrees FLOAT, sub2raining INTEGER"
readstream_df = spark.readStream.schema(schema).option("maxFilesPerTrigger", 1).parquet("hdfs://nn:9000/Rain_Forecast")

data = va.transform(readstream_df)
predictions = dt_model.transform(data)

In [31]:
#Use features as a source and apply your model to stream weather predictions to the screen
df = predictions.select("station", "date", "prediction").filter(col("station") == "A")
s = df.writeStream.format("console").trigger(processingTime="5 seconds").outputMode("append").start()
s.awaitTermination(20)
s.stop()

23/04/28 21:05:24 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-5c2dde42-81e9-4e3f-8e77-06e08f8a09d9. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/04/28 21:05:24 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+-------+----------+----------+
|station|      date|prediction|
+-------+----------+----------+
|      A|2000-01-10|       0.0|
|      A|2000-01-23|       0.0|
|      A|2000-01-26|       0.0|
|      A|2000-02-29|       0.0|
|      A|2000-03-12|       1.0|
|      A|2000-03-18|       0.0|
|      A|2000-03-21|       0.0|
|      A|2000-05-15|       1.0|
|      A|2000-05-16|       1.0|
|      A|2000-05-20|       0.0|
|      A|2000-05-22|       1.0|
|      A|2000-05-29|       0.0|
|      A|2000-06-16|       1.0|
|      A|2000-06-24|       1.0|
|      A|2000-06-27|       1.0|
|      A|2000-01-08|       0.0|
|      A|2000-02-18|       0.0|
|      A|2000-03-08|       1.0|
|      A|2000-03-14|       1.0|
|      A|2000-03-24|       0.0|
+-------+----------+----------+
only showing top 20 rows

-------------------------------------------
Batch: 1
-------------------------------------------
+-------+---